In [ ]:
#Calculate vorticity and divergence for 1 frame, and apply appropriate threshold for data

import pandas as pd
import numpy as np
from metpy.units import units
import metpy.calc as mpcalc
import matplotlib.pyplot as plt

# Read the data for the specified frame without headers
uwind = pd.read_excel('Example_Velocity_Map_X.xlsx', header=None)
vwind = pd.read_excel('Example_Velocity_Map_Y.xlsx', header=None)

# Convert velocity data to units
u = uwind.values * 5.86 * units('micrometer/min') #pull up data in units of distance/time
v = vwind.values * 5.86 * units('micrometer/min') #our conversion = (pixels/frame)(36000um/1228pixels)(1frame/5min) = 5.86

# Normalize the velocity
normV = np.sqrt(u**2 + v**2)

# Handle units manually during normalization
un = u / normV
vn = v / normV

# Define grid spacing (assuming a regular grid)
dx = 1 * units('micrometer')  # Replace with actual grid spacing
dy = 1 * units('micrometer')  # Replace with actual grid spacing

# Calculate vorticity & divergence
vorticity = mpcalc.vorticity(u, v, dx=dx, dy=dy)
divergence = mpcalc.divergence(u, v, dx=dx, dy=dy)

# Calculate normalized vorticity & divergence
vorticity_norm = normV * mpcalc.vorticity(un * normV.units, vn * normV.units, dx=dx, dy=dy)
divergence_norm = normV * mpcalc.divergence(un * normV.units, vn * normV.units, dx=dx, dy=dy)

# Apply threshold and set values within threshold range to NaN
threshold_low, threshold_high = min, max
vorticity_norm_magnitude = vorticity_norm.magnitude
divergence_norm_magnitude = divergence_norm.magnitude

vorticity_norm_magnitude[(vorticity_norm_magnitude > threshold_low) & (vorticity_norm_magnitude < threshold_high)] = np.nan
divergence_norm_magnitude[(divergence_norm_magnitude > threshold_low) & (divergence_norm_magnitude < threshold_high)] = np.nan

# Create mesh grid for plotting (assuming your data has shape [ny, nx])
ny, nx = u.shape
x = np.linspace(0, (nx-1)*dx.magnitude, nx)
y = np.linspace(0, (ny-1)*dy.magnitude, ny)
X, Y = np.meshgrid(x, y)

# Plot normalized vorticity
fig, ax = plt.subplots(figsize=(8, 6))

vmin, vmax = min, max  # Set the desired min and max values
im = ax.contourf(X, Y, vorticity_norm_magnitude, cmap='RdBu_r', levels=np.linspace(vmin, vmax, 21))

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Normalized Vorticity')
ax.set_title('Normalized Vorticity Frame 60\nPositive: Counterclockwise, Negative: Clockwise')
plt.show()

# Plot normalized divergence
fig, ax = plt.subplots(figsize=(8, 6))

vmin, vmax = min, max  # Set the desired min and max values
im = ax.contourf(X, Y, divergence_norm_magnitude, cmap='RdBu_r', levels=np.linspace(vmin, vmax, 21))

cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Normalized Divergence')
ax.set_title('Normalized Divergence Frame 60\nPositive: Diverging, Negative: Converging')
plt.show()

# Save normalized vorticity and divergence to Excel files
vorticity_norm_df = pd.DataFrame(vorticity_norm_magnitude)
divergence_norm_df = pd.DataFrame(divergence_norm_magnitude)

vorticity_norm_df.to_excel('norm_vorticity_thresh.xlsx', index=False, header=False)
divergence_norm_df.to_excel('norm_divergence_thresh.xlsx', index=False, header=False)

In [ ]:
#Determine min and max values of vorticity and divergence datasets for above cell vmin and vmax 

import pandas as pd
import numpy as np
from metpy.units import units
import metpy.calc as mpcalc

def read_velocity_data(frame_number):
    """Read the velocity data for a given frame number."""
    uwind = pd.read_excel(f'Velocity_Map_X_213_Frame_{frame_number}.xlsx', header=None)
    vwind = pd.read_excel(f'Velocity_Map_Y_213_Frame_{frame_number}.xlsx', header=None)
    
    # Convert velocity data to units
    u = uwind.values * 5.86 * units('micrometer/min')
    v = vwind.values * 5.86 * units('micrometer/min')
    
    return u, v

def compute_vorticity_and_divergence(u, v, dx, dy):
    """Compute the vorticity and divergence from u and v components."""
    vorticity = mpcalc.vorticity(u, v, dx=dx, dy=dy)
    divergence = mpcalc.divergence(u, v, dx=dx, dy=dy)
    return vorticity, divergence

def find_min_max_vorticity_and_divergence(frames):
    min_vorticity = np.inf * units('1/s')
    max_vorticity = -np.inf * units('1/s')
    min_divergence = np.inf * units('1/s')
    max_divergence = -np.inf * units('1/s')
    
    # Define grid spacing (assuming a regular grid)
    dx = 1 * units('micrometer')  # Replace with actual grid spacing if known
    dy = 1 * units('micrometer')  # Replace with actual grid spacing if known
    
    for frame in frames:
        u, v = read_velocity_data(frame)
        vorticity, divergence = compute_vorticity_and_divergence(u, v, dx, dy)
        
        min_vorticity_frame = np.min(vorticity)
        max_vorticity_frame = np.max(vorticity)
        min_divergence_frame = np.min(divergence)
        max_divergence_frame = np.max(divergence)
        
        min_vorticity = min(min_vorticity, min_vorticity_frame)
        max_vorticity = max(max_vorticity, max_vorticity_frame)
        min_divergence = min(min_divergence, min_divergence_frame)
        max_divergence = max(max_divergence, max_divergence_frame)
        
        print(f"Frame {frame}: Min Vorticity = {min_vorticity_frame:.4f}, Max Vorticity = {max_vorticity_frame:.4f}")
        print(f"Frame {frame}: Min Divergence = {min_divergence_frame:.4f}, Max Divergence = {max_divergence_frame:.4f}")
    
    return min_vorticity, max_vorticity, min_divergence, max_divergence

# Specify the frames to analyze
frames = [frame numbers]

# Find the minimum and maximum vorticity and divergence values
min_vorticity, max_vorticity, min_divergence, max_divergence = find_min_max_vorticity_and_divergence(frames)
print(f"Overall Min Vorticity: {min_vorticity:.4f}")
print(f"Overall Max Vorticity: {max_vorticity:.4f}")
print(f"Overall Min Divergence: {min_divergence:.4f}")
print(f"Overall Max Divergence: {max_divergence:.4f}")

In [ ]:
#Upsampling files to match size of selected velocity regions

import pandas as pd
from skimage.transform import resize

# Function to upsample a dataframe
def upsample_dataframe(filename, new_shape, output_filename):
    # Read the data
    data = pd.read_excel(filename, header=None).values
    
    # Upsample the data
    upsampled_data = resize(data, new_shape, mode='reflect', anti_aliasing=True)
    
    # Convert back to dataframe
    df_upsampled = pd.DataFrame(upsampled_data)
    
    # Save the upsampled dataframe to an Excel file
    df_upsampled.to_excel(output_filename, header=False, index=False)
    
    # Return the shape of the upsampled data
    return df_upsampled.shape

# Define the new shape
new_shape = (920, 1228)

# Upsample vorticity data
vorticity_file = 'norm_vorticity_thresh.xlsx'
upsampled_vorticity_file = 'upsampled_norm_vort_thresh.xlsx'
vorticity_shape = upsample_dataframe(vorticity_file, new_shape, upsampled_vorticity_file)

# Upsample divergence data
divergence_file = 'norm_divergence_thresh.xlsx'
upsampled_divergence_file = 'upsampled_norm_div_thresh.xlsx'
divergence_shape = upsample_dataframe(divergence_file, new_shape, upsampled_divergence_file)

# Print the shapes of the upsampled files
print(f"Shape of '{upsampled_vorticity_file}': {vorticity_shape[0]} rows x {vorticity_shape[1]} columns")
print(f"Shape of '{upsampled_divergence_file}': {divergence_shape[0]} rows x {divergence_shape[1]} columns")

In [ ]:
#Use mask to plot vorticity and divergence within selected regions only, and generate frequency distribution and CDF plots

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import cumfreq
from skimage.transform import resize

# Function to apply mask and plot
def apply_mask_and_plot(data_filename, mask_filename, output_filename, vmin, vmax, title, threshold_low, threshold_high, save_thresholded_filename=None):
    # Read the data
    data = pd.read_excel(data_filename, header=None).values
    mask = pd.read_excel(mask_filename, header=None).values

    # Check if data is loaded correctly
    if data.size == 0:
        print(f"No data loaded from {data_filename}")
        return
    if mask.size == 0:
        print(f"No mask loaded from {mask_filename}")
        return

    # Resize mask to match the shape of the data
    mask_resized = resize(mask, data.shape, mode='reflect', anti_aliasing=True)
    mask_resized = np.where(mask_resized > 0.5, 1, np.nan)  # Convert to binary mask with NaNs

    # Apply the mask
    masked_data = np.where(np.isnan(mask_resized), np.nan, data)

    # Convert to DataFrame for easy plotting
    masked_df = pd.DataFrame(masked_data)

    # Plot heatmap of masked data
    plt.figure(figsize=(10, 7))
    sns.heatmap(masked_df, annot=False, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title(title)
    plt.show()

    # Flatten the masked data and remove NaNs for plotting
    flattened_data = masked_data.flatten()
    flattened_data = flattened_data[~np.isnan(flattened_data)]

    # Apply threshold to remove values between threshold_low and threshold_high
    filtered_data = flattened_data[~((flattened_data > threshold_low) & (flattened_data < threshold_high))]

    # Check if filtered data is empty
    if filtered_data.size == 0:
        print(f"No data available after thresholding for {title}")
        return

    # Frequency distribution plot
    plt.figure(figsize=(10, 7))
    sns.histplot(filtered_data, bins=50, kde=False)
    plt.title(f'Frequency Distribution of {title}')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.show()

    # CDF plot
    res = cumfreq(filtered_data, numbins=50)
    x = res.lowerlimit + np.linspace(0, res.binsize * res.cumcount.size, res.cumcount.size)

    plt.figure(figsize=(10, 7))
    plt.plot(x, res.cumcount / len(filtered_data))
    plt.title(f'Cumulative Distribution Function (CDF) of {title}')
    plt.xlabel('Value')
    plt.ylabel('CDF')
    plt.grid(True)
    plt.show()

    # Save the masked data to a new Excel file
    masked_df.to_excel(output_filename, header=False, index=False)

    # Save the filtered (thresholded) data to a new Excel file if requested
    if save_thresholded_filename:
        filtered_df = pd.DataFrame(filtered_data, columns=['Values'])
        filtered_df.to_excel(save_thresholded_filename, index=False)
        print(f"Thresholded data saved to {save_thresholded_filename}")

# Define the min and max values for the color scale
vmin_vorticity, vmax_vorticity = -1, 1  # Adjust based on your data range
vmin_divergence, vmax_divergence = -1.9, 1.9  # Adjust based on your data range

# Apply mask and plot for vorticity with its threshold
apply_mask_and_plot(
    'upsampled_norm_vort_thresh.xlsx', 
    'speed_area_selection.xlsx', 
    'masked_norm_vort_thresh.xlsx', 
    vmin_vorticity, vmax_vorticity, 
    'Masked Normalized Vorticity', 
    min, max, #include threshold values if needed
    'masked_norm_vort_thresh_combinedplot.xlsx'
)

# Apply mask and plot for divergence with its threshold
apply_mask_and_plot(
    'upsampled_norm_div_thresh.xlsx', 
    'speed_area_selection.xlsx', 
    'masked_norm_div_thresh.xlsx', 
    vmin_divergence, vmax_divergence, 
    'Masked Normalized Divergence', 
    min, max, #include threshold values if needed
    'masked_norm_div_threshcombinedplot.xlsx'
)

# Function to check the size of the Excel files (number of cells)
def check_size(filename):
    data = pd.read_excel(filename, header=None)
    return data.shape

# Check the sizes of the new files
vort_size = check_size('masked_norm_vort_thresh.xlsx')
div_size = check_size('masked_norm_div_thresh.xlsx')
print(f"Size of masked vorticity file: {vort_size}")
print(f"Size of masked divergence file: {div_size}")

In [ ]:
#frequency distribution vorticity for selected regions only, all time points

# Load files to plot
file_names = ['masked_norm_vort_thresh_combinedplot_1.xlsx',
              'masked_norm_vort_thresh_combinedplot2.xlsx',
              'masked_norm_vort_thresh_combinedplot3.xlsx',
              'masked_norm_vort_thresh_combinedplot4.xlsx']

# Assign the corresponding times to your datasets
times = [timepoint values]

#optional: create data threshold to remove background if needed
threshold = 

#create a figure and axis object
fig, ax = plt.subplots(figsize=(6, 6))

#loop through each file and plot histogram with outline
for file_name in file_names:
    #read data from Excel file
    df = pd.read_excel(file_name)
    
    #flatten the DataFrame to a single column and apply threshold
    all_values = df.values.flatten()
    non_blank_values_vorticity = all_values[(~pd.isna(all_values)) & (all_values < threshold value) | (all_values > threshold value)]
    
    #plot histogram with outline and no fill
    ax.hist(non_blank_values_vorticity, bins=50, alpha=0.8, histtype='step', linewidth=2.5, label=file_name)

#set labels and title for plot
ax.set_xlabel('Vorticity Magnitude')
ax.set_ylabel('Frequency')
ax.set_title('Frequency Distribution of Vorticity')

#set axes limits and show plot
ax.set_xlim(min, max)
ax.set_ylim(0, max)
#plt.legend()
plt.show()

In [ ]:
#frequency distribution divergence for selected regions only, all time points

# Load files to plot
file_names = ['masked_norm_div_thresh_combinedplot1.xlsx',
              'masked_norm_div_thresh_combinedplot2.xlsx',
              'masked_norm_div_thresh_combinedplot3.xlsx',
              'masked_norm_div_thresh_combinedplot4.xlsx']

# Assign the corresponding times to your datasets
times = [timepoint values]

#optional: create data threshold to remove background if needed
threshold = 

#create a figure and axis object
fig, ax = plt.subplots(figsize=(6, 6))

#loop through each file and plot histogram with outline
for file_name in file_names:
    #read data from Excel file
    df = pd.read_excel(file_name)
    
    #flatten the DataFrame to a single column and apply threshold
    all_values = df.values.flatten()
    non_blank_values_vorticity = all_values[(~pd.isna(all_values)) & (all_values < threshold value) | (all_values > threshold value)]
    
    #plot histogram with outline and no fill
    ax.hist(non_blank_values_vorticity, bins=50, alpha=0.8, histtype='step', linewidth=2.5, label=file_name)

#set labels and title for plot
ax.set_xlabel('Divergence Magnitude')
ax.set_ylabel('Frequency')
ax.set_title('Frequency Distribution of Divergence')

#set axes limits and show plot
ax.set_xlim(min, max)
ax.set_ylim(0, max)
#plt.legend()
plt.show()

In [ ]:
#Count clockwise and counterclockwise vorticity values from each timepoint outside thresholded range

import pandas as pd
import numpy as np

# Define the threshold range if needed
lower_threshold = 
upper_threshold = 

# Load files to process
file_names = ['masked_norm_vort_thresh_combinedplot_1.xlsx',
              'masked_norm_vort_thresh_combinedplot2.xlsx',
              'masked_norm_vort_thresh_combinedplot3.xlsx',
              'masked_norm_vort_thresh_combinedplot4.xlsx']

# Assign the corresponding times to your datasets
times = [timepoint values]

# Initialize counters for overall counts
total_positive_count = 0
total_negative_count = 0

# Function to count positive and negative values outside the threshold range
def count_values_outside_threshold(df, lower_threshold, upper_threshold):
    # Get all numerical values from the DataFrame
    all_values = df.values.flatten()
    
    # Remove NaN values
    non_nan_values = all_values[~np.isnan(all_values)]
    
    # Filter values outside the threshold range
    outside_threshold = non_nan_values[(non_nan_values < lower_threshold) | (non_nan_values > upper_threshold)]
    
    # Count positive and negative values
    positive_count = np.sum(outside_threshold > 0)
    negative_count = np.sum(outside_threshold < 0)
    
    return positive_count, negative_count

# Iterate over each file and accumulate the counts
for file_name, time in zip(file_names, times):
    # Read the Excel file into a DataFrame
    df = pd.read_excel(file_name)
    
    # Count the values outside the threshold range
    positive_count, negative_count = count_values_outside_threshold(df, lower_threshold, upper_threshold)
    
    # Print the results for each frame
    print(f"Frame {time} hr:")
    print(f"Total positive values outside threshold: {positive_count}")
    print(f"Total negative values outside threshold: {negative_count}\n")
    
    # Accumulate the counts
    total_positive_count += positive_count
    total_negative_count += negative_count

# Print the overall counts
print("Overall counts for all frames:")
print(f"Total positive values outside threshold: {total_positive_count}")
print(f"Total negative values outside threshold: {total_negative_count}")

In [ ]:
#Count divergence and convergence values from each timepoint outside thresholded range

import pandas as pd
import numpy as np

# Define the threshold range if needed
lower_threshold = 
upper_threshold = 

# Load files to process
file_names = ['masked_norm_div_thresh_combinedplot1.xlsx',
              'masked_norm_div_thresh_combinedplot2.xlsx',
              'masked_norm_div_thresh_combinedplot3.xlsx',
              'masked_norm_div_thresh_combinedplot4.xlsx']

# Assign the corresponding times to your datasets
times = [timepoint values]

# Initialize counters for overall counts
total_positive_count = 0
total_negative_count = 0

# Function to count positive and negative values outside the threshold range
def count_values_outside_threshold(df, lower_threshold, upper_threshold):
    # Get all numerical values from the DataFrame
    all_values = df.values.flatten()
    
    # Remove NaN values
    non_nan_values = all_values[~np.isnan(all_values)]
    
    # Filter values outside the threshold range
    outside_threshold = non_nan_values[(non_nan_values < lower_threshold) | (non_nan_values > upper_threshold)]
    
    # Count positive and negative values
    positive_count = np.sum(outside_threshold > 0)
    negative_count = np.sum(outside_threshold < 0)
    
    return positive_count, negative_count

# Iterate over each file and accumulate the counts
for file_name, time in zip(file_names, times):
    # Read the Excel file into a DataFrame
    df = pd.read_excel(file_name)
    
    # Count the values outside the threshold range
    positive_count, negative_count = count_values_outside_threshold(df, lower_threshold, upper_threshold)
    
    # Print the results for each frame
    print(f"Frame {time} hr:")
    print(f"Total positive values outside threshold: {positive_count}")
    print(f"Total negative values outside threshold: {negative_count}\n")
    
    # Accumulate the counts
    total_positive_count += positive_count
    total_negative_count += negative_count

# Print the overall counts
print("Overall counts for all frames:")
print(f"Total positive values outside threshold: {total_positive_count}")
print(f"Total negative values outside threshold: {total_negative_count}")